# LIBRARIES

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.models import load_model
import ast, os

# LABELLING 

# ast - abstract syntax tree - analysis tool

# literal_eval - identifying data type of the content in a file by traversing through the nodes of the tree

In [2]:
#loading model into model parameter
model = load_model('currency_mobilenetmodel.h5')

#opening currency classes text file
f = open("mobilenet_currency_class_indices.txt", "r")

#reading the text file into labels parameter
labels = f.read()

#identifying data type of the content in a file by traversing through the nodes of the tree
labels = ast.literal_eval(labels)
#print(labels)
#interchanging and printing the values as label:class instead of class:label
final_labels = {v: k for k, v in labels.items()}
print(final_labels)

{0: '100_new_back_down', 1: '100_new_back_up', 2: '100_new_front_down', 3: '100_new_front_up', 4: '100_old_back_down', 5: '100_old_back_up', 6: '100_old_front_down', 7: '100_old_front_up', 8: '10_new_back_down', 9: '10_new_back_up', 10: '10_new_front_down', 11: '10_new_front_up', 12: '10_old_back_down', 13: '10_old_back_up', 14: '10_old_front_down', 15: '10_old_front_up', 16: '2000_new_back_down', 17: '2000_new_back_up', 18: '2000_new_front_down', 19: '2000_new_front_up', 20: '200_new_back_down', 21: '200_new_back_up', 22: '200_new_front_down', 23: '200_new_front_up', 24: '20_new_back_down', 25: '20_new_back_up', 26: '20_new_front_down', 27: '20_new_front_up', 28: '20_old_back_down', 29: '20_old_back_up', 30: '20_old_front_down', 31: '20_old_front_up', 32: '500_new_back_down', 33: '500_new_back_up', 34: '500_new_front_down', 35: '500_new_front_up', 36: '50_new_back_down', 37: '50_new_back_up', 38: '50_new_front_down', 39: '50_new_front_up', 40: '50_old_back_down', 41: '50_old_back_up',

# PREDICTING IMAGE

# load_img-loads image into PIL(Python Imaging Library) format-provides the python interpreter with image editing capabilities.

# asarray-converts input to array-returns an array

# np.expand_dims-expand shape of an array

# result_dict[final_labels[key]] = result[0][key] - mapping every value of result array to the classes in final_label dictionary (final_labels[key]-gives values of respective keys from dictionary)

# dict.items()-returns tuple of key value pairs

# sorted() -used to sort dictionary

# key - supports a inline function-here it returns item[1] as item which in this case probability values beside every class name({class-items[0]:prob-items[1]})

# based on item[1] the dictionary is sorted with reverse =true which is in descending order

# sorted_results.keys()-key values(class names) of sorted_result dict

# sorted_results[list(sorted_results.keys())[0]] - retriving the value of the first element in the list which contains key values

In [3]:
def predict_image(imgname, from_test_dir):
    test_image = tf.keras.utils.load_img(imgname, target_size = (224, 224))#image dim=224X224
    
    #converts input to array-returns an array
    test_image = np.asarray(test_image)
    
    #axis increase number of dimensions of array-axis=0-vertically-rows
    test_image = np.expand_dims(test_image, axis=0)
    
    #normalizing the values of images
    test_image = (2.0 / 255.0) * test_image - 1.0 
    
    #passing the normalized image to our model for prediction
    result = model.predict(test_image)
    
    #print(result)-returns an result array with probabilities of each class
    
    result_dict = dict() #creating a dictionary
    
    for key in list(final_labels.keys()):#each keys of final labels dict
        result_dict[final_labels[key]] = result[0][key]
    
    #sorting dictionary based on probability values in descending order
    sorted_results = {k: v for k, v in sorted(result_dict.items(), key=lambda item: item[1], reverse=True)} 

    if not from_test_dir: #if False is passed 
        for label in sorted_results.keys(): #each class name from sorted dictionary
           
    #printing each class name(key) and its value by multiplying value to 100 to achieve probability percentage in that format 
            print("{}: {}%".format(label, sorted_results[label] * 100))
            
    final_result = dict() #creating a dictionary 
    
    #creating and mapping the 1st key value pair into new dict
    final_result[list(sorted_results.keys())[0]] = sorted_results[list(sorted_results.keys())[0]] * 100
    return final_result

#path of test image passed to predict image()
note=predict_image('..\\test-images\\2.jpg',True)

#print(note.keys())#predicted class will be in dictionary
currency=list(note.keys())[0]#retriving that class name from dictionary

print("Currency note: ",currency)

1/1 [==============================] - 2s 2s/step
Currency note:  10_new_back_up


# VALIDATING BATCH-TEST DIRECTORY

# true-only prints class with highest probability
# false-prints all classes with probabilities
# calling predict image directory and returns result to prediction parameter
# prediction is returned in dictionary form hence we check key value in that dict with currency string

In [4]:
def verify_test_dir():
    #path of test directory
    path = '..\\batch-test-images'
    
    #list of all files(images) in test dir will be stored in files array
    files = os.listdir(path) 
    correct_preds = 0
    
    #total image count in test dir
    file_count = len(files) 
    finals = []
    
    for filename in files:
        #taking each name of file into filename parameter
        digits = []
        
        for char in filename:
            #taking each character of each file into char parameter
            
            #checking whether the character is digit or not
            if char.isdigit():
                digits += char #if digit then add to digits list
            else:
                break #else break

        num = "".join(digits) #join values in digits list-[1,0,0]-100

        currency_string = []
        currency_string.append(num)#append to currency string
        
        #checking what type of note it is
        if 'old' in filename:
            currency_string.append('old')
        else:
            currency_string.append('new')

        if 'back' in filename:
            currency_string.append('back')
        elif 'front' in filename:
            currency_string.append('front')

        #checking if the filename contains r(reverse allignment)-100back.jpg-[100back,jpg]
        rev = filename.split('.')[0][-1:]
        
        #print(rev)-[0]means [100back]-[-1] means k
        if rev == 'r': #k!=r
            currency_string.append('down')
        else:
            currency_string.append('up') #then 100back.jpg is up

        currency_string = "_".join(currency_string)#joining whole string with help of _

        prediction = predict_image(path + '\\' + filename, True)
        
        if list(prediction.keys())[0] == currency_string: 
            #filename along with its prediction probability
            print("{}: CORRECT PREDICTION".format(filename), prediction)
            
            print("Predicted note: ",list(prediction.keys())[0])#predicted note
            
            print("Orginal note:",currency_string)#orginal note
            
            correct_preds += 1 #incrementing correct prediction value
        else:
            print("{}: INCORRECT PREDICTION!".format(filename), prediction)
            print("Predicted note: ",list(prediction.keys())[0])
            print("Orginal note:", currency_string)
            
    acc=(correct_preds/file_count)*100 #accuracy
    print('accuracy:',acc)
    
verify_test_dir()

1/1 [==============================] - 0s 71ms/step
100back.jpg: CORRECT PREDICTION {'100_new_back_up': 56.30040168762207}
Predicted note:  100_new_back_up
Orginal note: 100_new_back_up
1/1 [==============================] - 0s 80ms/step
100backr.jpg: CORRECT PREDICTION {'100_new_back_down': 40.80900549888611}
Predicted note:  100_new_back_down
Orginal note: 100_new_back_down
1/1 [==============================] - 0s 73ms/step
100front.jpg: CORRECT PREDICTION {'100_new_front_up': 60.642510652542114}
Predicted note:  100_new_front_up
Orginal note: 100_new_front_up
1/1 [==============================] - 0s 87ms/step
100front7.jpg: CORRECT PREDICTION {'100_new_front_up': 48.73565435409546}
Predicted note:  100_new_front_up
Orginal note: 100_new_front_up
1/1 [==============================] - 0s 77ms/step
100frontr.jpg: INCORRECT PREDICTION! {'50_new_front_down': 17.689599096775055}
Predicted note:  50_new_front_down
Orginal note: 100_new_front_down
1/1 [==============================] - 0

1/1 [==============================] - 0s 62ms/step
20oldfrontr.jpg: CORRECT PREDICTION {'20_old_front_down': 44.74850594997406}
Predicted note:  20_old_front_down
Orginal note: 20_old_front_down
1/1 [==============================] - 0s 68ms/step
500back.jpg: CORRECT PREDICTION {'500_new_back_up': 84.37694907188416}
Predicted note:  500_new_back_up
Orginal note: 500_new_back_up
1/1 [==============================] - 0s 64ms/step
500back1.jpg: CORRECT PREDICTION {'500_new_back_up': 90.03133177757263}
Predicted note:  500_new_back_up
Orginal note: 500_new_back_up
1/1 [==============================] - 0s 65ms/step
500back1r.jpg: CORRECT PREDICTION {'500_new_back_down': 22.60117083787918}
Predicted note:  500_new_back_down
Orginal note: 500_new_back_down
1/1 [==============================] - 0s 67ms/step
500backr.jpg: CORRECT PREDICTION {'500_new_back_down': 34.12860631942749}
Predicted note:  500_new_back_down
Orginal note: 500_new_back_down
1/1 [==============================] - 0s 64

# COMPLETE CODE

In [9]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.models import load_model
import ast, os

model = load_model('currency_mobilenetmodel.h5')
f = open("mobilenet_currency_class_indices.txt", "r")
labels = f.read()
labels = ast.literal_eval(labels)
final_labels = {v: k for k, v in labels.items()}


def predict_image(imgname, from_test_dir):
    test_image = tf.keras.utils.load_img(imgname, target_size = (224, 224))

    # plt.imshow(test_image)
    # plt.show()

    test_image = np.asarray(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_image = (2.0 / 255.0) * test_image - 1.0
    result = model.predict(test_image)

    result_dict = dict()
    for key in list(final_labels.keys()):
        result_dict[final_labels[key]] = result[0][key]
    sorted_results = {k: v for k, v in sorted(result_dict.items(), key=lambda item: item[1], reverse=True)}

    if not from_test_dir:
        print('=' * 50)
        for label in sorted_results.keys():
            print("{}: {}%".format(label, sorted_results[label] * 100))

    final_result = dict()
    final_result[list(sorted_results.keys())[0]] = sorted_results[list(sorted_results.keys())[0]] * 100

    return final_result

def verify_test_dir():
    path = '..\\batch-test-images'
    files = os.listdir(path)
    correct_preds = 0
    file_count = len(files)
    finals = []
    for filename in files:

        digits = []
        for al in filename:
            if al.isdigit():
                digits += al
            else:
                break

        num = "".join(digits)

        final_string = []
        final_string.append(num)

        if 'old' in filename:
            final_string.append('old')
        else:
            final_string.append('new')

        if 'back' in filename:
            final_string.append('back')
        elif 'front' in filename:
            final_string.append('front')

        rev = filename.split('.')[0][-1:]
        if rev == 'r':
            final_string.append('down')
        else:
            final_string.append('up')

        final_string = "_".join(final_string)

        prediction = predict_image(path + '\\' + filename, True)
        if list(prediction.keys())[0] == final_string:
            print("{}: Correct Prediction".format(filename), prediction)
            print("Predicted note: ",list(prediction.keys())[0])
            print("Orginal note:",final_string)
            correct_preds += 1
        else:
            print("{}: INCORRECT PREDICTION!".format(filename), prediction)
            print("Predicted note: ",list(prediction.keys())[0])
            print("Orginal note:",final_string)
    acc=(correct_preds/file_count)*100
    print('accuracy:',acc)
print('=' * 50)
note=predict_image('..\\test-images\\kappa.jpg', True)
currency=list(note.keys())[0]
print("Currency note: ",currency)
verify_test_dir()

1/1 [==============================] - 1s 668ms/step
Currency note:  50_old_back_down
1/1 [==============================] - 0s 49ms/step
100back.jpg: Correct Prediction {'100_new_back_up': 50.23735761642456}
Predicted note:  100_new_back_up
Orginal note: 100_new_back_up
1/1 [==============================] - 0s 45ms/step
100backr.jpg: Correct Prediction {'100_new_back_down': 47.92151749134064}
Predicted note:  100_new_back_down
Orginal note: 100_new_back_down
1/1 [==============================] - 0s 49ms/step
100front.jpg: Correct Prediction {'100_new_front_up': 34.36532020568848}
Predicted note:  100_new_front_up
Orginal note: 100_new_front_up
1/1 [==============================] - 0s 49ms/step
100front7.jpg: Correct Prediction {'100_new_front_up': 32.50194489955902}
Predicted note:  100_new_front_up
Orginal note: 100_new_front_up
1/1 [==============================] - 0s 46ms/step
100frontr.jpg: Correct Prediction {'100_new_front_down': 29.828277230262756}
Predicted note:  100_new_